In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',100)
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from plotly.offline import iplot
import pandas as pd
import plotly.graph_objs as go
#import cufflinks
from plotly.offline import init_notebook_mode

In [2]:
customer_df = pd.read_excel('../Customer_Data.xlsx')
invoice_df = pd.read_csv('../Final_invoice.csv')
jtd_df = pd.read_csv('../JTD.csv')

In [3]:
invoice_df_2 = pd.read_csv('../Final_invoice.csv')

In [4]:
invoice_df_2.shape

(492314, 59)

In [5]:
invoice_df_2['Regn No'].nunique()

260323

In [6]:
print('Total Records in Customer Table:',customer_df.shape)
print('Total Records in Invoice Table:',invoice_df.shape)
print('Total Records in Inventory Table:',jtd_df.shape)

Total Records in Customer Table: (555338, 9)
Total Records in Invoice Table: (492314, 59)
Total Records in Inventory Table: (5619484, 10)


#### Dropped Unnecessary columns from Invoice Dataframe

In [11]:
invoice_df.drop(['Amt Rcvd From Custom','Amt Rcvd From Ins Co','CGST(14%)','CGST(2.5%)',
                 'CGST(6%)','CGST(9%)','IGST(12%)','IGST(18%)','IGST(28%)','IGST(5%)',
                 'Outstanding Amt','SGST/UGST(14%)', 'SGST/UGST(2.5%)', 'SGST/UGST(6%)', 
                 'SGST/UGST(9%)','Service Advisor Name', 'TDS amount','Total CGST', 'Total GST', 'Total IGST',
                 'Total SGST/UGST','Policy no.','Gate Pass Date','Cash /Cashless Type',
                 'Insurance Company','Claim No.','Expiry Date','Regn No'],axis =1 ,inplace=True)

In [12]:
invoice_df.drop(['Technician Name','Total Value','ODN No.','Area / Locality'],axis =1 ,inplace=True)

##### Created Service Time Feature from Invoice DateTime and Job Card DateTime

In [13]:
invoice_df["Invoice_DateTime"]=invoice_df["Invoice Date"]+' '+invoice_df["Invoice Time"]
invoice_df["JobCard_DateTime"]=invoice_df["JobCard Date"]+' '+invoice_df["JobCard Time"]
invoice_df['JobCard_DateTime'] = pd.to_datetime(invoice_df['JobCard_DateTime'],dayfirst=True)
invoice_df['Invoice_DateTime'] = pd.to_datetime(invoice_df['Invoice_DateTime'],dayfirst=True)
invoice_df["Service_Time"]=invoice_df["Invoice_DateTime"]-invoice_df["JobCard_DateTime"]

In [14]:
invoice_df.head()

,Unnamed: 0,CITY,Cust Type,Customer No.,District,Gate Pass Time,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Print Status,Recovrbl Exp,Total Amt Wtd Tax.,User ID,Invoice_DateTime,JobCard_DateTime,Service_Time
0,0,Thane,Retail,67849,Maharashtra,00:00:00,02-01-2015,7005200002,11:30:36,168303,13-12-2014,14:29:43,49317,1203.14,GENERAL MOTORS,0.00,SPARK,500.06,Paid Service,2348.75,400601,BC01,THANE,NO,0.0,4051.95,BC01FS1,2015-01-02 11:30:36,2014-12-13 14:29:43,19 days 21:00:53
1,1,THNAE,Retail,84419,Maharashtra,00:00:00,03-01-2015,7005200003,10:07:32,173997,02-01-2015,14:12:18,78584,804.26,TATA MOTORS,197.03,INDICA,0.00,SMC Value Package,0.00,400603,BC01,THANE,NO,0.0,1001.29,BC01SA2,2015-01-03 10:07:32,2015-01-02 14:12:18,0 days 19:55:14
2,2,THANE[W],Retail,81055,Maharashtra,00:00:00,03-01-2015,7005200004,11:12:57,173889,02-01-2015,11:40:44,33985,180.19,MARUTI SUZUKI,0.00,ZEN,0.00,Running Repairs,52.95,400607,BC01,THANE,NO,0.0,233.14,BC01SA2,2015-01-03 11:12:57,2015-01-02 11:40:44,0 days 23:32:13
3,3,THNAE,Retail,84419,Maharashtra,00:00:00,03-01-2015,7005200005,11:40:44,174158,03-01-2015,10:12:32,78584,0.00,TATA MOTORS,0.00,INDICA,0.00,SMC Redemption,0.00,400603,BC01,THANE,NO,0.0,0.00,BC01SA2,2015-01-03 11:40:44,2015-01-03 10:12:32,0 days 01:28:12
4,4,THANE,Retail,18980,Maharashtra,00:00:00,03-01-2015,7005200006,12:07:17,173860,02-01-2015,10:45:42,50057,1202.97,HYUNDAI,100.00,SANTRO XING,499.99,Paid Service,944.16,400603,BC01,THANE,NO,0.0,2747.12,BC01SA2,2015-01-03 12:07:17,2015-01-02 10:45:42,1 days 01:21:35


In [15]:
invoice_df.drop(['JobCard Date','JobCard Time','Invoice Time'],axis = 1,inplace=True)

In [16]:
invoice_df.isnull().sum().sort_values(ascending=False)

Model                 1525
CITY                     1
Misc Total               0
Cust Type                0
Customer No.             0
District                 0
Gate Pass Time           0
Invoice Date             0
Invoice No               0
Job Card No              0
KMs Reading              0
Labour Total             0
Make                     0
Service_Time             0
JobCard_DateTime         0
OSL Total                0
Order Type               0
Parts Total              0
Pin code                 0
Plant                    0
Plant Name1              0
Print Status             0
Recovrbl Exp             0
Total Amt Wtd Tax.       0
User ID                  0
Invoice_DateTime         0
Unnamed: 0               0
dtype: int64

In [17]:
invoice_df.District.value_counts().head()

Maharashtra       128119
Tamil Nadu        116309
Karnataka          44135
Andhra Pradesh     40604
Uttar Pradesh      31534
Name: District, dtype: int64

In [18]:
invoice_df.Make.value_counts().head(10)

MARUTI SUZUKI           147102
MAHINDRA &  MAHINDRA     87356
HYUNDAI                  85342
TATA MOTORS              58034
GENERAL MOTORS           29092
FORD                     21616
TOYOTA                   19466
HONDA                    15464
SKODA                     6515
FIAT                      6437
Name: Make, dtype: int64

In [193]:
invoice_df.Model.value_counts().head(20)

SWIFT           31899
ALTO            28569
SANTRO          24536
SCORPIO CRDE    23771
WAGONR          22864
I10             21892
INDICA          19810
SWIFT DZIRE     16004
BOLERO          15909
XYLO            13346
I20             12896
INNOVA          11470
CITY            10665
VISTA           10341
INDIGO          10337
800             10185
VERNA            8564
FIGO             7709
FIESTA           7667
LOGAN D          7353
Name: Model, dtype: int64

In [194]:
invoice_df['Cust Type'].value_counts()

Retail              488995
Fleets                2218
Corporate others       579
Corporate- M&M         254
MFCWL                  252
Franchise               15
Insurance                1
Name: Cust Type, dtype: int64

In [39]:
invoice_df['Order Type'].value_counts()

Running Repairs      269455
Paid Service         135004
Accidental            44159
SMC Redemption        15738
Mechanical            15100
SMC Value Package      9781
Repeat Order           3043
WBW Order                17
Workshop Damage          17
Name: Order Type, dtype: int64

In [19]:
cat_cols = invoice_df.select_dtypes(include='object').columns
num_cols = invoice_df.select_dtypes(exclude='object').columns

In [20]:
print(cat_cols)
print(num_cols)

Index(['CITY', 'Cust Type', 'Customer No.', 'District', 'Gate Pass Time',
       'Invoice Date', 'Make', 'Model', 'Order Type', 'Plant', 'Plant Name1',
       'Print Status', 'User ID'],
      dtype='object')
Index(['Unnamed: 0', 'Invoice No', 'Job Card No', 'KMs Reading',
       'Labour Total', 'Misc Total', 'OSL Total', 'Parts Total', 'Pin code',
       'Recovrbl Exp', 'Total Amt Wtd Tax.', 'Invoice_DateTime',
       'JobCard_DateTime', 'Service_Time'],
      dtype='object')


In [21]:
invoice_df['Plant'].value_counts().head(10)

BC03    28615
BC01    14483
BC05    13323
BC26    13253
BC06    12752
BC07    11815
BC20    10259
BC08    10146
BC13     9990
BC23     9521
Name: Plant, dtype: int64

In [22]:
invoice_df[invoice_df['Plant']=='BC03']['District'].mode()[0]

'Maharashtra'

In [23]:
invoice_df['Plant'].nunique()

242

In [24]:
plant = list(invoice_df['Plant'].unique())

In [25]:
plant

['BC01',
 'BC02',
 'BC03',
 'BC05',
 'BC06',
 'BC07',
 'BC08',
 'BC12',
 'BC13',
 'BC15',
 'BC16',
 'BC17',
 'BC18',
 'BC19',
 'BC20',
 'BC21',
 'BC22',
 'BC23',
 'BC24',
 'BC25',
 'BC26',
 'BC27',
 'BC28',
 'BC29',
 'BC30',
 'BC31',
 'BC32',
 'BC33',
 'BC34',
 'X001',
 'X005',
 'X006',
 'X008',
 'X010',
 'X011',
 'X012',
 'X013',
 'X014',
 'X017',
 'X020',
 'X021',
 'X022',
 'X024',
 'X025',
 'X029',
 'X034',
 'X036',
 'X037',
 'X039',
 'X041',
 'X016',
 'X018',
 'X026',
 'X028',
 'X049',
 'X007',
 'X027',
 'X060',
 'X023',
 'X031',
 'X032',
 'X042',
 'X045',
 'X052',
 'X054',
 'X061',
 'X072',
 'X033',
 'X051',
 'X085',
 'X098',
 'X055',
 'X058',
 'X068',
 'X074',
 'X081',
 'X048',
 'X057',
 'X082',
 'X110',
 'X064',
 'X075',
 'X080',
 'X035',
 'X047',
 'X062',
 'X063',
 'X092',
 'X109',
 'X056',
 'X102',
 'X103',
 'X124',
 'X003',
 'X044',
 'X066',
 'X079',
 'X101',
 'X114',
 'X126',
 'X127',
 'X128',
 'X174',
 'X175',
 'BC04',
 'BC11',
 'X046',
 'X238',
 'X125',
 'X136',
 'X091',
 

### Distrcit Names given were incorrect as a single plant was to shown in many districts. Found out the District mode for each plant

In [26]:
for i in plant:
     print(i+ ' : ',invoice_df[invoice_df['Plant']==i]['District'].mode()[0])

BC01 :  Maharashtra
BC02 :  Maharashtra
BC03 :  Maharashtra
BC05 :  Andhra Pradesh
BC06 :  Tamil Nadu
BC07 :  Tamil Nadu
BC08 :  Gujarat
BC12 :  Andhra Pradesh
BC13 :  Gujarat
BC15 :  Karnataka
BC16 :  Tamil Nadu
BC17 :  Andhra Pradesh
BC18 :  Tamil Nadu
BC19 :  Maharashtra
BC20 :  Uttar Pradesh
BC21 :  Kerala
BC22 :  Uttar Pradesh
BC23 :  Haryana
BC24 :  Maharashtra
BC25 :  Maharashtra
BC26 :  Maharashtra
BC27 :  Punjab
BC28 :  Uttar Pradesh
BC29 :  Uttar Pradesh
BC30 :  Punjab
BC31 :  Karnataka
BC32 :  Punjab
BC33 :  Haryana
BC34 :  Maharashtra
X001 :  Maharashtra
X005 :  Tamil Nadu
X006 :  Maharashtra
X008 :  Puducherry
X010 :  Tamil Nadu
X011 :  Tamil Nadu
X012 :  Tamil Nadu
X013 :  Tamil Nadu
X014 :  Tamil Nadu
X017 :  Maharashtra
X020 :  Tamil Nadu
X021 :  Tamil Nadu
X022 :  Tamil Nadu
X024 :  Tamil Nadu
X025 :  Punjab
X029 :  Punjab
X034 :  Maharashtra
X036 :  Tamil Nadu
X037 :  Andhra Pradesh
X039 :  Tamil Nadu
X041 :  Uttar Pradesh
X016 :  Maharashtra
X018 :  Maharashtra
X026 

##### Created a new State column and mapped the above dictionary of plants and states there are present in. Cross Verfied many plant codes in plant master table and they showed the same states

In [27]:
invoice_df.loc[:,'state'] =  invoice_df.loc[:,'Plant'].map({'BC01':'Maharashtra',
'BC02':'Maharashtra',
'BC03':'Maharashtra',
'BC05':'Andhra Pradesh',
'BC06':'Tamil Nadu',
'BC07':'Tamil Nadu',
'BC08':'Gujarat',
'BC12':'Andhra Pradesh',
'BC13':'Gujarat',
'BC15':'Karnataka',
'BC16':'Tamil Nadu',
'BC17':'Andhra Pradesh',
'BC18':'Tamil Nadu',
'BC19':'Maharashtra',
'BC20':'Uttar Pradesh',
'BC21':'Kerala',
'BC22':'Uttar Pradesh',
'BC23':'Haryana',
'BC24':'Maharashtra',
'BC25':'Maharashtra',
'BC26':'Maharashtra',
'BC27':'Punjab',
'BC28':'Uttar Pradesh',
'BC29':'Uttar Pradesh',
'BC30':'Punjab',
'BC31':'Karnataka',
'BC32':'Punjab',
'BC33':'Haryana',
'BC34':'Maharashtra',
'X001':'Maharashtra',
'X005':'Tamil Nadu',
'X006':'Maharashtra',
'X008':'Puducherry',
'X010':'Tamil Nadu',
'X011':'Tamil Nadu',
'X012':'Tamil Nadu',
'X013':'Tamil Nadu',
'X014':'Tamil Nadu',
'X017':'Maharashtra',
'X020':'Tamil Nadu',
'X021':'Tamil Nadu',
'X022':'Tamil Nadu',
'X024':'Tamil Nadu',
'X025':'Punjab',
'X029':'Punjab',
'X034':'Maharashtra',
'X036':'Tamil Nadu',
'X037':'Andhra Pradesh',
'X039':'Tamil Nadu',
'X041':'Uttar Pradesh',
'X016':'Maharashtra',
'X018':'Maharashtra',
'X026':'Maharashtra',
'X028':'Maharashtra',
'X049':'Tamil Nadu',
'X007':'Karnataka',
'X027':'Maharashtra',
'X060':'Madhya Pradesh',
'X023':'Tamil Nadu',
'X031':'Tamil Nadu',
'X032':'Tamil Nadu',
'X042':'Chhattisgarh',
'X045':'Haryana',
'X052':'Tamil Nadu',
'X054':'Andhra Pradesh',
'X061':'Uttarakhand',
'X072':'Punjab',
'X033':'Karnataka',
'X051':'Tamil Nadu',
'X085':'Tamil Nadu',
'X098':'Himachal Pradesh',
'X055':'Andhra Pradesh',
'X058':'Haryana',
'X068':'Andhra Pradesh',
'X074':'Maharashtra',
'X081':'Kerala',
'X048':'Tamil Nadu',
'X057':'Karnataka',
'X082':'Telangana',
'X110':'Tamil Nadu',
'X064':'Haryana',
'X075':'Uttar Pradesh',
'X080':'Maharashtra',
'X035':'Maharashtra',
'X047':'Tamil Nadu',
'X062':'Karnataka',
'X063':'Andhra Pradesh',
'X092':'Madhya Pradesh',
'X109':'Tamil Nadu',
'X056':'Andhra Pradesh',
'X102':'Karnataka',
'X103':'Rajasthan',
'X124':'Kerala',
'X003':'Karnataka',
'X044':'Tamil Nadu',
'X066':'Tamil Nadu',
'X079':'Karnataka',
'X101':'Maharashtra',
'X114':'Madhya Pradesh',
'X126':'Uttar Pradesh',
'X127':'Madhya Pradesh',
'X128':'Tamil Nadu',
'X174':'Telangana',
'X175':'Telangana',
'BC04':'Andhra Pradesh',
'BC11':'Maharashtra',
'X046':'Haryana',
'X238':'Maharashtra',
'X125':'Madhya Pradesh',
'X136':'Rajasthan',
'X091':'Madhya Pradesh',
'X167':'Rajasthan',
'X171':'Rajasthan',
'X106':'Madhya Pradesh',
'X233':'Uttarakhand',
'X084':'Karnataka',
'X111':'Punjab',
'X118':'Madhya Pradesh',
'X147':'Karnataka',
'X279':'Gujarat',
'X156':'Madhya Pradesh',
'X209':'Uttar Pradesh',
'X095':'Rajasthan',
'X112':'Bihar',
'X123':'Bihar',
'X164':'Rajasthan',
'X113':'Karnataka',
'X154':'Tamil Nadu',
'X105':'Tamil Nadu',
'X146':'Maharashtra',
'X204':'Andhra Pradesh',
'X215':'Bihar',
'X229':'Maharashtra',
'X086':'Himachal Pradesh',
'X099':'Maharashtra',
'X138':'Himachal Pradesh',
'X170':'Rajasthan',
'X173':'Maharashtra',
'X185':'Rajasthan',
'X186':'Karnataka',
'X187':'Maharashtra',
'X192':'Kerala',
'X224':'Uttar Pradesh',
'X267':'Madhya Pradesh',
'X107':'Maharashtra',
'X245':'West Bengal',
'X129':'Haryana',
'X140':'Karnataka',
'X252':'Bihar',
'X297':'Maharashtra',
'X328':'Gujarat',
'X050':'Maharashtra',
'X053':'Uttar Pradesh',
'X219':'Madhya Pradesh',
'X259':'Odisha',
'X076':'Uttar Pradesh',
'X172':'Rajasthan',
'X190':'Karnataka',
'X331':'Tamil Nadu',
'X341':'Rajasthan',
'X188':'Bihar',
'X218':'Maharashtra',
'X243':'Uttarakhand',
'X249':'Tamil Nadu',
'X254':'Tamil Nadu',
'X264':'Punjab',
'X386':'Telangana',
'X405':'Tamil Nadu',
'X132':'Uttar Pradesh',
'X201':'Uttar Pradesh',
'X247':'Odisha',
'X327':'Gujarat',
'X347':'Gujarat',
'X155':'Madhya Pradesh',
'X270':'West Bengal',
'X322':'Odisha',
'X329':'Gujarat',
'X401':'Gujarat',
'X149':'Maharashtra',
'X217':'Maharashtra',
'X253':'Maharashtra',
'X260':'Rajasthan',
'X350':'Gujarat',
'X364':'Odisha',
'X121':'Karnataka',
'X122':'Bihar',
'X437':'Maharashtra',
'X073':'Maharashtra',
'X261':'Haryana',
'X139':'Bihar',
'X241':'Uttarakhand',
'X242':'Uttarakhand',
'X246':'Odisha',
'X435':'Madhya Pradesh',
'X133':'Tamil Nadu',
'X166':'Madhya Pradesh',
'X180':'Tamil Nadu',
'X183':'Maharashtra',
'X203':'Andhra Pradesh',
'X277':'West Bengal',
'X334':'Rajasthan',
'X418':'Bihar',
'X281':'Rajasthan',
'X317':'Rajasthan',
'X308':'Jharkhand',
'X413':'Uttar Pradesh',
'X205':'Uttar Pradesh',
'X265':'Rajasthan',
'X332':'Kerala',
'X366':'Odisha',
'X384':'Tamil Nadu',
'X392':'Haryana',
'X135':'Maharashtra',
'X177':'Rajasthan',
'X199':'Chhattisgarh',
'X244':'Rajasthan',
'X339':'Odisha',
'X444':'Maharashtra',
'X131':'Uttar Pradesh',
'X169':'Rajasthan',
'X286':'Bihar',
'X357':'Uttar Pradesh',
'X380':'Uttar Pradesh',
'X320':'Assam',
'X161':'Chandigarh',
'X324':'Tamil Nadu',
'X397':'Gujarat',
'X089':'Maharashtra',
'X214':'Uttar Pradesh',
'X269':'West Bengal',
'X316':'Rajasthan',
'X083':'Telangana',
'X119':'Bihar',
'X196':'Maharashtra',
'X202':'Andhra Pradesh',
'X403':'Uttar Pradesh',
'X458':'Andhra Pradesh',
'X206':'Uttar Pradesh',
'X359':'West Bengal',
'X432':'Uttarakhand',
'X285':'Rajasthan',
'X211':'Uttar Pradesh',
})


In [28]:
invoice_df['state'].isnull().sum()

0

In [29]:
invoice_df.state.value_counts()

Maharashtra         130210
Tamil Nadu          114350
Karnataka            44306
Andhra Pradesh       42367
Uttar Pradesh        35973
Gujarat              25901
Punjab               23989
Haryana              20171
Madhya Pradesh       13688
Rajasthan            13346
Telangana             5932
Kerala                5750
Himachal Pradesh      3974
Bihar                 3390
Uttarakhand           2628
Odisha                2327
Puducherry            1779
West Bengal           1306
Chhattisgarh           598
Jharkhand              177
Assam                   92
Chandigarh              60
Name: state, dtype: int64

In [30]:
invoice_df['District'].value_counts()

Maharashtra             128119
Tamil Nadu              116309
Karnataka                44135
Andhra Pradesh           40604
Uttar Pradesh            31534
Gujarat                  24403
Punjab                   22741
Haryana                  17971
Madhya Pradesh           13598
Rajasthan                13466
Telangana                 7741
Kerala                    6002
Delhi                     5414
Chandigarh                3319
Himachal Pradesh          3316
Bihar                     3258
Uttarakhand               2541
Odisha                    2305
Puducherry                1716
West Bengal               1332
Dadra and Nagar Hav.       907
Chhattisgarh               622
Daman and Diu              436
Jharkhand                  256
Assam                      100
Arunachal Pradesh           62
Nagaland                    24
Goa                         24
Jammu and Kashmir           21
Andaman and Nico.In.         8
Lakshadweep                  7
Megalaya                     7
Sikkim  

##### Created a new City column and mapped the above dictionary of plants and cities there are present in. 

In [31]:
for i in plant:
     print(i+ ' : ',invoice_df[invoice_df['Plant']==i]['CITY'].mode()[0])

BC01 :  THANE
BC02 :  PUNE
BC03 :  NASIK
BC05 :  HYDERABAD
BC06 :  CHENNAI
BC07 :  CHENNAI
BC08 :  VAPI
BC12 :  HYDERABAD
BC13 :  SURAT
BC15 :  BENGALURU
BC16 :  CHENNAI
BC17 :  NELLORE
BC18 :  COIMBATORE
BC19 :  PUNE
BC20 :  SAHIBABAD
BC21 :  ERNAKULAM
BC22 :  GREATER NOIDA
BC23 :  faridabad
BC24 :  PUNE
BC25 :  NAVI MUMBAI
BC26 :  NAGPUR
BC27 :  LUDHIANA
BC28 :  NOIDA
BC29 :  NOIDA
BC30 :  MOHALI
BC31 :  BANGALORE
BC32 :  LUDHIANA
BC33 :  panchkula
BC34 :  PUNE
X001 :  THANE
X005 :  CHENNAI
X006 :  miraroad
X008 :  PUDUCHERRY
X010 :  RANIPET
X011 :  villupuram
X012 :  chidambaram
X013 :  NEYVELI
X014 :  AMBUR
X017 :  KOLHAPUR
X020 :  COIMBATORE
X021 :  Salem
X022 :  THANJAVUR
X024 :  trichy
X025 :  MOGA
X029 :  CHANDIGARH
X034 :  AURANGABAD
X036 :  MADURAI
X037 :  KURNOOL
X039 :  chennai
X041 :  MEERUT
X016 :  PUNE
X018 :  SHIRUR
X026 :  KHED
X028 :  A.NAGAR
X049 :  TUTICORIN
X007 :  BANGALORE
X027 :  TALEGAON
X060 :  CWA
X023 :  TIRUPUR
X031 :  POLLACHI
X032 :  DHARMAPURI
X042 :  RA

In [32]:
Plant_df = pd.read_excel('Plant Master.xlsx')

In [33]:
City_dict = dict(zip(Plant_df.Plant, Plant_df.City))

In [34]:
City_dict

{'BC07': 'Chennai',
 'BC30': 'Mohali',
 'X001': 'Thane',
 'X002': 'Ghaziabad',
 'X003': 'Bengaluru',
 'X005': 'Chennai',
 'X006': 'Thane',
 'X007': 'Bengaluru',
 'X008': 'Puducherry',
 'X010': 'Ranipet',
 'X011': 'Villupuram',
 'X012': 'Chidambaram',
 'X013': 'Neyveli',
 'X014': 'Ambur',
 'X016': 'Boripardhi',
 'X017': 'Kolhapur',
 'X018': 'Shirur',
 'X020': 'Coimbatore',
 'X021': 'Salem',
 'X022': 'Thanjavur',
 'X023': 'Tirupur',
 'X024': 'Tiruchirappalli',
 'X025': 'Moga',
 'X026': 'Pune',
 'X027': 'Pune',
 'X028': 'Ahmednagar',
 'X029': 'Zirakpur',
 'X031': 'Pollachi',
 'X032': 'Dharmapuri',
 'X033': 'Shimoga',
 'X034': 'Aurangabad',
 'X035': 'Pune',
 'X036': 'Madurai',
 'X037': 'Kurnool',
 'X039': 'Chennai',
 'X041': 'Meerut',
 'X042': 'Raipur',
 'X044': 'Dindigul',
 'X045': 'Rohtak',
 'X046': 'Jhajjar',
 'X047': 'Tirunelveli',
 'X048': 'Nagercoil',
 'X049': 'Tuticorin',
 'X050': 'Vadali Bhoi',
 'X051': 'Coimbatore',
 'X052': 'Madurai',
 'X053': 'Bhagpat',
 'X054': 'Ongole',
 'X055

In [35]:
def clean_cityData(data):
    if 'X' in data:
        data = City_dict.get(data)
    return data

In [36]:
invoice_df['City_clean'] = invoice_df['Plant'].apply(lambda x: clean_cityData(x))

In [40]:
invoice_df['City_clean'].unique()

array(['thane', 'pune', 'nashik', 'hyderabad', 'chennai', 'vapi', 'surat',
       'bengaluru', 'nellore', 'coimbatore', 'sahibabad', 'ernakulam',
       'greater noida', 'faridabad', 'navi mumbai', 'nagpur', 'ludhiana',
       'noida', 'mohali', 'bangalore', 'panchkula', 'puducherry',
       'ranipet', 'villupuram', 'chidambaram', 'neyveli', 'ambur',
       'kolhapur', 'salem', 'thanjavur', 'tiruchirappalli', 'moga',
       'zirakpur', 'aurangabad', 'madurai', 'kurnool', 'meerut',
       'boripardhi', 'shirur', 'ahmednagar', 'tuticorin', 'chhindwara',
       'tirupur', 'pollachi', 'dharmapuri', 'raipur', 'rohtak', 'ongole',
       'rishikesh', 'abohar', 'shimoga', 'chengalpattu', 'una',
       'rajahmundry', 'sonepat', 'thirupathi', 'shrigonda', 'kottayam',
       'nagercoil', 'hassan', 'khammam', 'hisar', 'agra', 'tirunelveli',
       'mysore', 'anantapur', 'indore', 'vijayawada', 'mangalore',
       'chomu', 'kochi', 'dindigul', 'palani', 'kolar', 'indapur',
       'bhopal', 'varanas

In [38]:
invoice_df.loc[:,'City_clean'] =  invoice_df.loc[:,'City_clean'].replace({'BC01':'THANE',
'BC02':'PUNE',
'BC03':'NASHIK',
'BC04':'BALANAGAR',                                                                         
'BC05':'HYDERABAD',
'BC06':'CHENNAI',
'BC07':'CHENNAI',
'BC08':'VAPI',
'BC11':'ANDHERI',
'BC12':'HYDERABAD',
'BC13':'SURAT',
'BC15':'BENGALURU',
'BC16':'CHENNAI',
'BC17':'NELLORE',
'BC18':'COIMBATORE',
'BC19':'PUNE',
'BC20':'SAHIBABAD',
'BC21':'ERNAKULAM',
'BC22':'GREATER NOIDA',
'BC23':'faridabad',
'BC24':'PUNE',
'BC25':'NAVI MUMBAI',
'BC26':'NAGPUR',
'BC27':'LUDHIANA',
'BC28':'NOIDA',
'BC29':'NOIDA',
'BC30':'MOHALI',
'BC31':'BANGALORE',
'BC32':'LUDHIANA',
'BC33':'panchkula',
'BC34':'PUNE'})

In [39]:
invoice_df['City_clean']  = invoice_df['City_clean'].str.lower()

##### There were around 1325 null values in model column. Imputed the same with mode for each make respectively

In [41]:
mask =invoice_df['Model'].isnull()

In [42]:
invoice_df.loc[mask,'Model'] = invoice_df.loc[mask,'Make'].map({'MAHINDRA &  MAHINDRA' : 'SCORPIO CRDE','FORD':'FIESTA','HONDA':'CITY',
                                                  'HYUNDAI':'SANTRO','TATA MOTORS':'INDICA','TOYOTA':'INNOVA','FIAT':'GRAND PUNTO',
                                                   'MARUTI SUZUKI':'SWIFT','GENERAL MOTORS':'SPARK','NISSAN':'MICRA','SKODA':'FABIA',
                                                   'VOLKSWAGEN':'POLO','RENAULT':'DUSTER','BMW':'5 SERIES'})

In [44]:
invoice_df.drop(['CITY','District'],axis =1,inplace=True)

In [45]:
invoice_df.drop(['Unnamed: 0'],axis =1,inplace=True)

In [46]:
invoice_df.drop(['Pin code'],axis =1,inplace=True)

In [47]:
invoice_df.rename(columns={"City_clean": "City"},inplace=True)

In [51]:
invoice_df.isnull().sum()

Cust Type             0
Customer No.          0
Gate Pass Time        0
Invoice Date          0
Invoice No            0
Job Card No           0
KMs Reading           0
Labour Total          0
Make                  0
Misc Total            0
Model                 0
OSL Total             0
Order Type            0
Parts Total           0
Plant                 0
Plant Name1           0
Print Status          0
Recovrbl Exp          0
Total Amt Wtd Tax.    0
User ID               0
Invoice_DateTime      0
JobCard_DateTime      0
Service_Time          0
state                 0
City                  0
Zones                 0
dtype: int64

In [49]:
invoice_df.state.unique()

array(['Maharashtra', 'Andhra Pradesh', 'Tamil Nadu', 'Gujarat',
       'Karnataka', 'Uttar Pradesh', 'Kerala', 'Haryana', 'Punjab',
       'Puducherry', 'Madhya Pradesh', 'Chhattisgarh', 'Uttarakhand',
       'Himachal Pradesh', 'Telangana', 'Rajasthan', 'Bihar',
       'West Bengal', 'Odisha', 'Jharkhand', 'Assam', 'Chandigarh'],
      dtype=object)

#### Bucketed States into Zones. Have made the north east zone to east zone as nort east had only Assam state

In [50]:
invoice_df['Zones']=invoice_df.state.replace(
['Karnataka','Andhra Pradesh','Tamil Nadu','Telangana','Kerala','Puducherry'],'South').replace(
['Gujarat','Maharashtra'],'West').replace(
['Bihar', 'Jharkhand','West Bengal','Odisha','Assam'],'East').replace(
['Punjab','Haryana','Chandigarh','Himachal Pradesh','Rajasthan'],'North').replace(
['Uttar Pradesh','Madhya Pradesh','Chhattisgarh','Uttarakhand'],'Centre')

In [52]:
invoice_df['Year'] = pd.to_datetime(invoice_df["Invoice_DateTime"]).dt.year
invoice_df['Month'] = pd.to_datetime(invoice_df["Invoice_DateTime"]).dt.month

In [53]:
invoice_df["Season"] = invoice_df["Month"].replace([10,11,12,1],"Winter").replace([2,3,4,5],"Summer").replace([6,7,8,9],"Monsoon")

In [61]:
invoice_df.to_csv('clean_invoice_2.csv')

#### Merging Customer and Invoice Dataframes

In [62]:
invoice_df = pd.read_csv('clean_invoice_2.csv')

In [63]:
customer_df = pd.read_excel('Customer_Data.xlsx')

In [65]:
age_df = pd.read_csv('new_full_data1.csv')

In [66]:
invoice_df['ServiceHours']=invoice_df.Service_Time.apply(lambda x:x[0:2]).astype(int)*24 + invoice_df.Service_Time.apply(lambda x:x[7:10]).str.replace(':','').str.replace('s','').astype(int)+round(invoice_df.Service_Time.apply(lambda x:x[10:13]).str.replace(':','').astype(float)/60,2)
invoice_df['ServiceHours']=invoice_df['ServiceHours'].replace(0.0,0.1)

In [67]:
invoice_df.columns

Index(['Unnamed: 0', 'Cust Type', 'Customer No.', 'Gate Pass Time',
       'Invoice Date', 'Invoice No', 'Job Card No', 'KMs Reading',
       'Labour Total', 'Make', 'Misc Total', 'Model', 'OSL Total',
       'Order Type', 'Parts Total', 'Plant', 'Plant Name1', 'Print Status',
       'Recovrbl Exp', 'Total Amt Wtd Tax.', 'User ID', 'Invoice_DateTime',
       'JobCard_DateTime', 'Service_Time', 'state', 'City', 'Zones', 'Year',
       'Month', 'Season', 'ServiceHours'],
      dtype='object')

In [68]:
invoice_df = pd.merge(invoice_df,age_df[['Invoice No','age']],on='Invoice No',how='left')

In [69]:
invoice_df[['Days','Last']] = invoice_df.Service_Time.apply( 
   lambda x: pd.Series(str(x).split(" ",1))) 

In [70]:
invoice_df['Days'] = invoice_df['Days'].astype(int)

In [71]:
customer_df.drop(['Title', 'Marital Status', 'Occupation', 'Date of Birth', 'Death date'],axis =1 ,inplace=True)

In [72]:
customer_df['Partner Type'] = customer_df['Partner Type'].fillna(customer_df['Partner Type'].mode()[0])
customer_df['Data Origin'] = customer_df['Data Origin'].fillna(customer_df['Data Origin'].mode()[0])

In [73]:
customer_df.isna().sum()

Business Partner    0
Customer No.        0
Partner Type        0
Data Origin         0
dtype: int64

In [74]:
customer_df['Customer No.'] = customer_df['Customer No.'].astype(str)

In [75]:
invoice_df['Customer No.'] = invoice_df['Customer No.'].astype(str)

In [76]:
cust_inv = pd.merge(invoice_df,customer_df,on ='Customer No.',how='left')

In [77]:
cust_inv.drop('Last',axis=1,inplace=True)

In [78]:
cust_inv.drop('Gate Pass Time',axis=1,inplace=True)
cust_inv.drop('Business Partner',axis=1,inplace=True)

In [80]:
cust_inv['Partner Type'] = cust_inv['Partner Type'].fillna(customer_df['Partner Type'].mode()[0])
cust_inv['Data Origin'] = cust_inv['Data Origin'].fillna(customer_df['Data Origin'].mode()[0])

In [82]:
cust_inv.to_csv('Customer_invoice_cleaned.csv')

#### Creating Model Categories from Model Price.

In [83]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [84]:
CarMake_List = ['Hyundai','Maruti','Tata','Toyota','Honda','Fiat','Ford','Skoda','Mahindra','Volkswagen','Nissan','Renault','Mercedes-Benz','Mitsubishi','BMW','Force',
               'Volvo','Audi','Premier','Porsche','Land_Rover','Jeep','MG']

In [85]:
rows = []
for carsMakes in CarMake_List:
    base_url = "https://www.cardekho.com/cars/" + carsMakes
    headers = ({'User-Agent':
                'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit\
    /537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'})
    response = get(base_url,headers=headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    tag = html_soup.find_all(class_='card shadowWPadding')
    for tags in tag:  
        if 'lakh' in tags.find_all(class_='price')[0].text.lower() :
            amt = 100000
        else:
            amt = 10000000
        if 'Rs.' in tags.find_all(class_='price')[0].text :
            CarPrice = int(float(tags.find_all(class_='price')[0].text.split('Rs.')[1].split()[0].strip())*amt)
        else:
            CarPrice = int(float(tags.find_all(class_='price')[0].text.split('Rs')[1].split()[0].strip())*amt)
        rows.append([tags.find_all('a')[0].text, CarPrice])

In [86]:
rows

[['Hyundai Creta', 999000],
 ['Hyundai Elite i20', 649000],
 ['Hyundai Venue', 670000],
 ['Hyundai Grand i10', 589000],
 ['Hyundai Verna', 930000],
 ['Hyundai Aura', 579000],
 ['Hyundai Santro', 457000],
 ['Hyundai Xcent', 581000],
 ['Hyundai Elantra', 1760000],
 ['Hyundai Tucson', 1876000],
 ['Hyundai Grand i10 Nios', 505999],
 ['Hyundai Kona Electric', 2375000],
 ['Hyundai Tucson 2020', 2000000],
 ['Hyundai Elite i20 2020', 560000],
 ['Hyundai Ioniq', 2000000],
 ['Hyundai Sonata', 2077000],
 ['Hyundai Palisade', 4000000],
 ['Maruti Swift', 519000],
 ['Maruti Baleno', 563000],
 ['Maruti Dzire', 589000],
 ['Maruti Vitara Brezza', 734000],
 ['Maruti Wagon R', 445000],
 ['Maruti Ertiga', 759000],
 ['Maruti S-Presso', 370000],
 ['Maruti Alto 800', 294000],
 ['Maruti Celerio', 441000],
 ['Maruti Ciaz', 831000],
 ['Maruti S-Cross', 880000],
 ['Maruti Ignis', 488999],
 ['Maruti XL6', 984000],
 ['Maruti Eeco', 380000],
 ['Maruti Celerio X', 490000],
 ['Maruti Ertiga Sport', 830000],
 ['Maruti

In [87]:
df_Cars = pd.DataFrame(rows, columns=["CarName", "CarPrice"])

In [88]:
df_Cars.head()

,CarName,CarPrice
0,Hyundai Creta,999000
1,Hyundai Elite i20,649000
2,Hyundai Venue,670000
3,Hyundai Grand i10,589000
4,Hyundai Verna,930000


In [89]:
df_Cars[['Make','Model']] = df_Cars.CarName.apply( 
   lambda x: pd.Series(str(x).split(" ",1))) 

In [90]:
df_Cars.Model = df_Cars.Model.str.upper()

In [91]:
Car_price_dict = dict(zip(df_Cars.Model, df_Cars.CarPrice))

In [92]:
Car_price_dict

{'CRETA': 999000,
 'ELITE I20': 649000,
 'VENUE': 670000,
 'GRAND I10': 589000,
 'VERNA': 930000,
 'AURA': 579000,
 'SANTRO': 457000,
 'XCENT': 581000,
 'ELANTRA': 1760000,
 'TUCSON': 1876000,
 'GRAND I10 NIOS': 505999,
 'KONA ELECTRIC': 2375000,
 'TUCSON 2020': 2000000,
 'ELITE I20 2020': 560000,
 'IONIQ': 2000000,
 'SONATA': 2077000,
 'PALISADE': 4000000,
 'SWIFT': 519000,
 'BALENO': 563000,
 'DZIRE': 589000,
 'VITARA BREZZA': 734000,
 'WAGON R': 445000,
 'ERTIGA': 759000,
 'S-PRESSO': 370000,
 'ALTO 800': 294000,
 'CELERIO': 441000,
 'CIAZ': 831000,
 'S-CROSS': 880000,
 'IGNIS': 488999,
 'XL6': 984000,
 'EECO': 380000,
 'CELERIO X': 490000,
 'ERTIGA SPORT': 830000,
 'S-CROSS 2020': 900000,
 'SWIFT 2020': 520000,
 'XL5': 500000,
 'SWIFT HYBRID': 1000000,
 'ALTROZ': 529000,
 'TIAGO': 459999,
 'NEXON': 695000,
 'HARRIER': 1369000,
 'TIGOR': 575000,
 'YODHA PICKUP': 694000,
 'TIGOR EV': 953999,
 'NEXON EV': 1399000,
 'GRAVITAS': 1500000,
 'TIAGO EV': 600000,
 'HBX': 500000,
 'HEXA': 140

In [93]:
cust_inv_df = pd.read_csv('Customer_invoice_cleaned.csv')

In [94]:
cust_inv_df.Model.nunique()

245

In [95]:
def fetch_price(model):
    price = [val for key, val in Car_price_dict.items() if model in key]
    if not price:
        return 0
    else:
        return price[0]

In [96]:
cust_inv_df['Price'] =  cust_inv_df['Model'].apply(lambda x :fetch_price(x.split(' ')[0]))

In [97]:
mask_df = cust_inv_df[cust_inv_df['Price']==0]

In [98]:
mask_df.groupby('Make')['Model'].unique()

Make
AUDI                                                             [Q5, A4]
BMW                               [BMW-M3, BMW-M5, BMW-M6, MINIS, BMW X6]
DAEWOO                                              [MATIZ, CIELO, NEXIA]
FORCE                                                   [FORCE ONE, TRAX]
FORD                    [FIESTA, IKON, FUSION, ESCORT, FOCUS, MONDEO, ...
GENERAL MOTORS          [SPARK, AVEO, TAVERA, BEAT, OPTRA, CHEVROLET C...
HINDUSTAN MOTORS          [CONTESSA, AMBASSADOR, AVIGO, TREKKER, PUSHPAK]
HONDA                                        [ACCORD, MOBILIO, SAIL, BRV]
HYUNDAI                 [ACCENT, GETZ, EON, TERRACAN, SANTA FE, VELOSTER]
MAHINDRA &  MAHINDRA    [XYLO, LOGAN P, LOGAN D, MAXX, QUANTO, SSANYON...
MARUTI SUZUKI           [ZEN, WAGONR, ESTEEM 1300, ZEN ESTILO, RITZ, S...
MERCEDES BENZ           [ML 350, SLK-CLASS, CL-CLASS, CLS-CLASS, SL-CL...
MITSUBISHI MOTORS                                [LANCER, MONTERO, CEDIA]
MORRIS                           

#### Some olde car prices couls not be scraped as they had been discontinued. Hence mapped them by creating aidctionary as below

In [99]:
old_Car_dict= {
'Q5' :5600000 ,
'A4' :4100000,
'BMW-M3':13000000,
'BMW-M5' :15000000,
'BMW-M6' :17600000, 
'MINIS' :3900000,
'BMW':9500000,
'MATIZ' :400000,
'CIELO' :655000,
'NEXIA' :622000,
'PALIO' :358000 ,
'ADVENTURE' :711000 ,
'UNO' :422000,
'PETRA' :508000,
'SIENA' :525000,
'FORCE':872000 ,
'TRAX' :703000,
'FIESTA' :850000,
'IKON' :482000 ,
'FUSION' :633000 ,
'ESCORT' :900000 ,
'FOCUS' :900000,
'MONDEO' :813000,
'LIMOUSINE':80000000,
'SPARK' :360000 ,
'AVEO' :650000 ,
'TAVERA' :928000 ,
'BEAT' :432000,
'OPTRA' :861000,
'CHEVROLET' :1395000,
'OPEL' :439000,
'ENJOY' :534000 ,
'SAIL' :439000 ,
'CAPTIVA' :2513000,
'SRV' :698000,
'TAVERA' :855000 ,
'FORESTER' :1000000 ,
'VECTRA' :1675000 ,
'TRAILBLAZER' :2640000,
'CONTESSA' :483000,
'AMBASSADOR' :485000,
'AVIGO' :469000,
'TREKKER' :488000,
'PUSHPAK' :349000,
'ACCORD' :4500000,
'MOBILIO' :717000,
'BRV' :952000,
'ACCENT' :673000,
'GETZ' :399000,
'EON' :395000,
'TERRACAN' :2500000,
'SANTA' :2800000,
'VELOSTER' :1500000,
'XYLO' :917000 ,
'LOGAN' :491000,
'MAXX' :799000,
'QUANTO' :599000 ,
'SSANYONG' :2057000 ,
'COMMANDER' :608000, 
'ARMADA' :650000,
'MAHINDRA':650000,
'MARSHAL' :599000 ,
'CLASSIC' :647000 ,  
'REVA' :355000 ,
'MAXXIMO':309000, 
'GENIO' :648000,
'TUV300' :854000 ,
'SUPRO' :530000 ,
'ZEN' :340000,
'WAGONR' :451000,
'RITZ' :506000,
'SX4' :715000,
'OMNI' :215000,
'ESTEEM' :463000,
'VERSA' :430000,
'GYPSY' :651000,
'KIZASHI' :1698000,
'ESTILO' :318000,
'ASTAR' :371000,
'ML' :6500000, 
'SLK-CLASS' :6500000 ,
'CL-CLASS' :6500000 ,
'CLS-CLASS' :6500000 ,
'SL-CLASS' :6500000,
'ML-CLASS' :6500000,
'CLK-CLASS' :6500000 ,
'GL-CLASS' :6500000 ,
'LANCER' :733000 ,
'MONTERO' :5798000,
'CEDIA' :824000,
'AUSTIN' :1500000,
'MICRA' :619000,
'X-TRAIL' :3200000,
'TEANA' :2114000,
'LAUREL' :1500000 ,
'TERRANO' :999000,
'DATSUN' :399000 ,
'EVALIA' :878000,
'CAYMAN' :8500000,
'RENAULT':812000,
'PULSE' :577000,
'FLUENCE' :1472000, 
'KOLEOS' :2392000, 
'LODGY' :999000,
'LAURA' :1766000, 
'FABIA' :631000,
'ROOMSTER' :1400000,
'YETI':2300000,
'RHINO' :759000,
'INDICA' :499000,
'INDIGO' :572000,
'VISTA' :467000,
'SAFARI' :1100000,
'SUMO' :580000,
'NANO' :236000,
'MAGIC' :450000,
'ARIA' :1143000,
'WINGER' :790000,
'VENTURE' :483000 ,
'SIERRA' :1400000,
'207' :525000,
'XENON':995000,
'JAGUAR' :10000000 ,
'ZEST' :582000 ,
'BOLT' :590000,
'ETIOS' :800000,
'QUALIS' :410000,
'TATA' :262000 ,
'TOUAREG' :5800000 ,
'AMEO' :532000,
'VOLVO' :4400000, 
}

In [100]:
def fetch_old_price(model):
    price = [val for key, val in old_Car_dict.items() if model in key]
    if not price:
        return 0
    else:
        return price[0]

In [101]:
mask_df['Price'] =  mask_df['Model'].apply(lambda x :fetch_old_price(x.split(' ')[0]))

In [102]:
mask_df[mask_df['Price']==0]

,Unnamed: 0,Unnamed: 0.1,Cust Type,Customer No.,Invoice Date,Invoice No,Job Card No,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Plant,Plant Name1,Print Status,Recovrbl Exp,Total Amt Wtd Tax.,User ID,Invoice_DateTime,JobCard_DateTime,Service_Time,state,City,Zones,Year,Month,Season,ServiceHours,age,Days,Partner Type,Data Origin,Price
95551,95551,95551,Retail,128995,05-10-2015,7154201174,265094,3732,130.66,TOYOTA,0.0,TATA-NANO,0.0,Running Repairs,0.00,X021,X021 - THE TRUE SAI MOTORSS,NO,0.0,130.66,X021SA1,2015-10-05 15:46:28,2015-10-05 10:44:36,0 days 05:01:52.000000000,Tamil Nadu,salem,South,2015,10,Winter,5.02,<1,0,1.0,Z019,0
95601,95601,95601,Retail,128995,15-10-2015,7154201224,268732,3780,157.93,TOYOTA,0.0,TATA-NANO,0.0,Running Repairs,144.05,X021,X021 - THE TRUE SAI MOTORSS,NO,0.0,301.98,X021SA1,2015-10-15 18:56:21,2015-10-15 13:33:06,0 days 05:23:15.000000000,Tamil Nadu,salem,South,2015,10,Winter,5.38,<1,0,1.0,Z019,0


In [103]:
mask = (cust_inv_df['Price']==0)

In [104]:
cust_inv_df.loc[mask,'Price'] =  cust_inv_df.loc[mask,'Model'].apply(lambda x :fetch_old_price(x.split(' ')[0]))

In [105]:
cust_inv_df['Price'].replace({0 : 236000},inplace=True)

In [106]:
cust_inv_df[cust_inv_df['Model']=='TATA-NANO']

,Unnamed: 0,Unnamed: 0.1,Cust Type,Customer No.,Invoice Date,Invoice No,Job Card No,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Plant,Plant Name1,Print Status,Recovrbl Exp,Total Amt Wtd Tax.,User ID,Invoice_DateTime,JobCard_DateTime,Service_Time,state,City,Zones,Year,Month,Season,ServiceHours,age,Days,Partner Type,Data Origin,Price
95551,95551,95551,Retail,128995,05-10-2015,7154201174,265094,3732,130.66,TOYOTA,0.0,TATA-NANO,0.0,Running Repairs,0.00,X021,X021 - THE TRUE SAI MOTORSS,NO,0.0,130.66,X021SA1,2015-10-05 15:46:28,2015-10-05 10:44:36,0 days 05:01:52.000000000,Tamil Nadu,salem,South,2015,10,Winter,5.02,<1,0,1.0,Z019,236000
95601,95601,95601,Retail,128995,15-10-2015,7154201224,268732,3780,157.93,TOYOTA,0.0,TATA-NANO,0.0,Running Repairs,144.05,X021,X021 - THE TRUE SAI MOTORSS,NO,0.0,301.98,X021SA1,2015-10-15 18:56:21,2015-10-15 13:33:06,0 days 05:23:15.000000000,Tamil Nadu,salem,South,2015,10,Winter,5.38,<1,0,1.0,Z019,236000


In [112]:
cust_inv_df7.columns

Index(['Invoice No', 'Job Card No', 'Customer No.', 'Cust Type',
       'Partner Type', 'Order Type', 'Data Origin', 'User ID', 'Plant',
       'Plant Name1', 'City', 'state', 'Zones', 'Make', 'Model', 'Price',
       'age', 'Days', 'ServiceHours', 'Service_Time', 'Invoice Date',
       'KMs Reading', 'Year', 'Month', 'Season', 'Labour Total', 'Parts Total',
       'OSL Total', 'Misc Total', 'Recovrbl Exp', 'Total Amt Wtd Tax.'],
      dtype='object')

In [109]:
cust_inv_df7=cust_inv_df[['Invoice No','Job Card No','Customer No.','Cust Type','Partner Type','Order Type','Data Origin','User ID',
                         'Plant','Plant Name1','City','state','Zones',
                         'Make','Model','Price','age','Days','ServiceHours','Service_Time','Invoice Date',
                         'KMs Reading','Year', 'Month', 'Season',
                         'Labour Total','Parts Total','OSL Total','Misc Total','Recovrbl Exp','Total Amt Wtd Tax.']]

In [110]:
cust_inv_df.drop(['Unnamed: 0', 'Unnamed: 0.1','Invoice_DateTime', 'JobCard_DateTime','Print Status'],axis =1, inplace=True)

In [113]:
cust_inv_df7['Price'].describe()

count    4.923140e+05
mean     6.911873e+05
std      5.559432e+05
min      1.200000e+05
25%      4.570000e+05
50%      5.390000e+05
75%      8.300000e+05
max      8.000000e+07
Name: Price, dtype: float64

In [114]:
pd.qcut(cust_inv_df7['Price'], q=[0,.5123,.89,.9863,1])

0         (119999.999, 539000.0]
1         (119999.999, 539000.0]
2         (119999.999, 539000.0]
3         (119999.999, 539000.0]
4         (119999.999, 539000.0]
                   ...          
492309    (119999.999, 539000.0]
492310     (539000.0, 1143000.0]
492311    (1143000.0, 2500000.0]
492312     (539000.0, 1143000.0]
492313    (119999.999, 539000.0]
Name: Price, Length: 492314, dtype: category
Categories (4, interval[float64]): [(119999.999, 539000.0] < (539000.0, 1143000.0] < (1143000.0, 2500000.0] < (2500000.0, 80000000.0]]

In [115]:
bin_labels_5 = ['Budget Friendly', 'Mid Range', 'High Range', 'Luxury']
cust_inv_df7['Model Category']= pd.qcut(cust_inv_df7['Price'],
                              q=[0,.5123,.89,.9863,1],
                              labels=bin_labels_5)
cust_inv_df7.head()

,Invoice No,Job Card No,Customer No.,Cust Type,Partner Type,Order Type,Data Origin,User ID,Plant,Plant Name1,City,state,Zones,Make,Model,Price,age,Days,ServiceHours,Service_Time,Invoice Date,KMs Reading,Year,Month,Season,Labour Total,Parts Total,OSL Total,Misc Total,Recovrbl Exp,Total Amt Wtd Tax.,Model Category
0,7005200002,168303,67849,Retail,1.0,Paid Service,Z007,BC01FS1,BC01,THANE,thane,Maharashtra,West,GENERAL MOTORS,SPARK,360000,>5,19,477.00,19 days 21:00:53.000000000,02-01-2015,49317,2015,1,Winter,1203.14,2348.75,500.06,0.00,0.0,4051.95,Budget Friendly
1,7005200003,173997,84419,Retail,1.0,SMC Value Package,Z001,BC01SA2,BC01,THANE,thane,Maharashtra,West,TATA MOTORS,INDICA,499000,>5,0,19.92,0 days 19:55:14.000000000,03-01-2015,78584,2015,1,Winter,804.26,0.00,0.00,197.03,0.0,1001.29,Budget Friendly
2,7005200004,173889,81055,Retail,1.0,Running Repairs,Z005,BC01SA2,BC01,THANE,thane,Maharashtra,West,MARUTI SUZUKI,ZEN,340000,>5,0,23.53,0 days 23:32:13.000000000,03-01-2015,33985,2015,1,Winter,180.19,52.95,0.00,0.00,0.0,233.14,Budget Friendly
3,7005200005,174158,84419,Retail,1.0,SMC Redemption,Z001,BC01SA2,BC01,THANE,thane,Maharashtra,West,TATA MOTORS,INDICA,499000,>5,0,1.47,0 days 01:28:12.000000000,03-01-2015,78584,2015,1,Winter,0.00,0.00,0.00,0.00,0.0,0.00,Budget Friendly
4,7005200006,173860,18980,Retail,1.0,Paid Service,Z005,BC01SA2,BC01,THANE,thane,Maharashtra,West,HYUNDAI,SANTRO XING,457000,>5,1,25.35,1 days 01:21:35.000000000,03-01-2015,50057,2015,1,Winter,1202.97,944.16,499.99,100.00,0.0,2747.12,Budget Friendly


In [ ]:
cust_inv_df7.to_csv('cust_inv_merged_new2.csv')